# Supermarkets data cleaning

We are importing the data from ... . An Ironhack graduate who scraped data from the main 6 supermarkets in Berlin. The goal of this document is to import the data and clean it for our model. 

In [1]:
import pandas as pd
import numpy as np

## Clean data

- Translate data to english
- Isolate the package size
- Check values
- Normalize Unit

- Create a unique list with unique products of all different markets
- Analyse it
- Conclude how to approach the solution

### 1. Import data and explore

In [2]:
# Importing the data from excel
supermarkets_en = {
    'aldinorth': pd.read_excel('data/products-en/aldinorth-products-en.xls', index_col=0),
    'aldisouth': pd.read_excel('data/products-en/aldisouth-products-en.xls', index_col=0),
    'edeka': pd.read_excel('data/products-en/edeka-products-en.xls', index_col=0),
    #'kaufland': pd.read_csv('data/products-en/kaufland-products-en.csv', index_col=0),
    #'lidl': pd.read_csv('data/products-en/lidl-products-en.csv', index_col=0),
    #'rewe': pd.read_csv('data/products-en/rewe-product-en.csv', index_col=0)
}

In [3]:
supermarkets_en['edeka']

,Name,Price,Unit,Pack Size,Supermarket,Comparable Price
0,Coke,1.39,100 ml,Pet bottle 1.5 L,edeka,0.093
1,Nutella,1.77,100 gram,"""Glass 500 grams",edeka,0.354
2,Becel Gold 70% fat,1.49,100 gram,Cup 250 grams,edeka,0.6
3,Iglo fish fingers,2.99,100 gram,"""15-pack 450 grams""",edeka,0.6639999999999999
4,Good & Cheap Landgasthof Goulash Pan,1.99,100 gram,edeka,0.398,NaN
...,...,...,...,...,...,...
5043,Nivea Extreme Stay Lipstick 17 Extrem Pinkini,11.49,Price per piece,Pack of 1,edeka,11.49
5044,Nivea Extreme Stay Lipstick 21 Extreme Cinnam,11.49,Price per piece,Pack of 1,edeka,11.49
5045,Nivea Extreme Stay Lipstick 11 Extremely Beige,11.49,Price per piece,Pack of 1,edeka,11.49
5046,Nivea Turbo Color Nail Polish 12 Turbo Red,7.89,Price per piece,Bottle glass 1 piece,edeka,7.89


In [4]:
supermarkets_en['edeka'].dtypes

Name                object
Price               object
Unit                object
Pack Size           object
Supermarket         object
Comparable Price    object
dtype: object

For the moment, we are going to focus in the columns 'Name' and 'Price'

In [5]:
supermarkets_en['edeka']['Name'].value_counts()

Zentis breakfast jam extra raspberry, with 50% fruit content      2
MM extra dry                                                      2
Kühne Salatfix Creme Fraiche                                      2
Knorr Schlemmersauce Garlic Sauce                                 2
Thomy mustard sweeter                                             2
                                                                 ..
Danone Orchard Vanilla Peach Passion Fruit, 4 x 125 g             1
EDEKA Natural California almonds, whole kernels                   1
good & cheap fish salmon salmon                                   1
Hengstenberg 13 herb vinegar 5% acid                              1
Kelloggs Smacks Choco Tresor cereal cushion with choco filling    1
Name: Name, Length: 5014, dtype: int64

### 2. Clean data
#### 2.1 Remove duplicates

In [6]:
# Removing duplicate products
supermarkets_en['edeka'] = supermarkets_en['edeka'].drop_duplicates('Name')

In [7]:
supermarkets_en['edeka']['Name'].value_counts()

Seven star tomato ketchup                                         1
Feldmann naturally cloudy apple cider vinegar                     1
Rügenwalder Pomeranian Gutsleberwurst roughly                     1
Weiss witch house                                                 1
Knorr Sauce Pur Fine sauce, 4 x 0.25 L                            1
                                                                 ..
Iglo Asia sweet and sour chicken                                  1
Homann Dips & Creams Garlic Dip                                   1
Appel sugar color                                                 1
Tekrum nut corners                                                1
Kelloggs Smacks Choco Tresor cereal cushion with choco filling    1
Name: Name, Length: 5014, dtype: int64

In [8]:
# Removing duplicate products for each DataFrame
for market in supermarkets_en:
    supermarkets_en[market] = supermarkets_en[market].drop_duplicates('Name')

#### 2.2 Price to numeric

In [9]:
# Price to numeric
supermarkets_en['edeka']['Price'].value_counts()

0.99     376
1.99     354
2.99     202
1.49     198
1.29     179
        ... 
2.48       1
12.45      1
1.7        1
3.88       1
3.7        1
Name: Price, Length: 241, dtype: int64

In [10]:
# Let's check if there are null values
supermarkets_en['edeka']['Price'].isna().sum()

0

In [11]:
supermarkets_en['edeka']['Price'].head(50)

0         1.39
1         1.77
2         1.49
3         2.99
4         1.99
5         2.29
6          0.3
7         2.29
8         0.99
9         3.79
10        1.15
11        3.29
12        1.77
13        0.99
14        3.29
15        1.29
16        1.29
17        2.99
18        0.95
19        2.19
20        3.49
21        2.69
22        1.59
23        2.29
24        0.69
25        0.89
26        1.69
27        1.99
28        0.85
29        0.99
30        0.77
31        1.59
32        2.79
33        0.89
34        2.19
35        0.66
36        1.29
37        2.99
38    100 gram
39      100 ml
40    100 gram
41        2.49
42    100 gram
43    100 gram
44      100 ml
45        0.87
46       14.79
47        0.85
48        2.69
49        1.99
Name: Price, dtype: object

In [12]:
# count how many values contain 'grams'
supermarkets_en['edeka']['Price'].str.contains('gram').sum()

82

In [13]:
# Drop those rows with wrong values within the 'Price' column
supermarkets_en['edeka'] = supermarkets_en['edeka'][~supermarkets_en['edeka']['Price'].str.contains('gram')]

In [14]:
# To numeric
supermarkets_en['edeka']['Price'] = pd.to_numeric(supermarkets_en['edeka']['Price'], errors='coerce')

In [15]:
supermarkets_en['edeka']['Price'].value_counts()

0.99    376
1.99    354
2.99    202
1.49    198
1.29    179
       ... 
8.75      1
3.77      1
8.90      1
4.44      1
5.98      1
Name: Price, Length: 239, dtype: int64

In [16]:
# To numeric for each DataFrame
for market in supermarkets_en:
    supermarkets_en[market]['Price'] = pd.to_numeric(supermarkets_en[market]['Price'], errors='coerce')

In [17]:
supermarkets_en['edeka'].dtypes

Name                 object
Price               float64
Unit                 object
Pack Size            object
Supermarket          object
Comparable Price     object
dtype: object

#### 2.3 Drop columns that we do not need

In [18]:
supermarkets_en['edeka'].head()

,Name,Price,Unit,Pack Size,Supermarket,Comparable Price
0,Coke,1.39,100 ml,Pet bottle 1.5 L,edeka,0.093
1,Nutella,1.77,100 gram,"""Glass 500 grams",edeka,0.354
2,Becel Gold 70% fat,1.49,100 gram,Cup 250 grams,edeka,0.6
3,Iglo fish fingers,2.99,100 gram,"""15-pack 450 grams""",edeka,0.6639999999999999
4,Good & Cheap Landgasthof Goulash Pan,1.99,100 gram,edeka,0.398,NaN


In [19]:
supermarkets_en['edeka'].drop(columns=['Supermarket', 'Comparable Price', 'Unit', 'Pack Size'], inplace=True)

In [20]:
# To numeric for each DataFrame
for market in supermarkets_en:
    supermarkets_en[market].drop(columns=['Supermarket', 'Comparable Price', 'Unit', 'Pack Size', 'Pack size', 'Comparable price'], inplace=True, errors='ignore')

In [21]:
supermarkets_en['edeka'].head()

,Name,Price
0,Coke,1.39
1,Nutella,1.77
2,Becel Gold 70% fat,1.49
3,Iglo fish fingers,2.99
4,Good & Cheap Landgasthof Goulash Pan,1.99


For now, we are going to proceed with this data. Possible improvements for the future can be:
    - Normalize the sizes of the products
    - Add the rest of the supermarkets
    - Add more columns

### 3. Export data

In [22]:
for market in supermarkets_en:
        supermarkets_en[market].to_pickle(f'data/products-clean/{market}-products-clean.pkl')